<a href="https://colab.research.google.com/github/filitesfam/React-Laravel-with-Restful-API/blob/main/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ktrain

     |████████████████████████████████| 25.3MB 129kB/s 
     |████████████████████████████████| 6.8MB 56.4MB/s 
     |████████████████████████████████| 983kB 54.3MB/s 
     |████████████████████████████████| 266kB 55.6MB/s 
     |████████████████████████████████| 1.9MB 44.5MB/s 
     |████████████████████████████████| 1.2MB 47.6MB/s 
     |████████████████████████████████| 471kB 50.9MB/s 
     |████████████████████████████████| 3.3MB 47.5MB/s 
     |████████████████████████████████| 870kB 36.8MB/s 
  Created wheel for ktrain: filename=ktrain-0.26.2-cp37-none-any.whl size=25277794 sha256=e9b42bfa102b195c4a17dc05c1a98e74b4eeb3ac132932e9cddf2adc7dead59b
  Stored in directory: /root/.cache/pip/wheels/67/2e/f1/c72afa08df8b2d984b910dea228902ce81dae4511afe9fafd2
  Created wheel for langdetect: filename=langdetect-1.0.8-cp37-none-any.whl size=993193 sha256=29aa6a1b5c83854081d11f1247bbfb89cb7983b17dbb9839bfc6f5e341b59a50
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d4

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import ktrain
from ktrain import text
import tensorflow as tf
from sklearn.model_selection import train_test_split
import sklearn

In [ ]:
data=pd.read_csv('/content/drive/MyDrive/combined.csv')
data = data.dropna(axis=0, subset=['Review'])
data = data[['Review','rate']]
data.loc[data["rate"] == 0, "rate"] = "negative" 
data.loc[data["rate"] == 1, "rate"] = "positive" 
data['rate'].value_counts()

positive    86492
negative    27419
Name: rate, dtype: int64

In [ ]:
total_negative =data.copy()
total_negative = total_negative.loc[total_negative['rate'] == 'negative']
total_negative = total_negative[:3000]
train_neg,test_neg = train_test_split(total_negative,train_size=0.5,random_state=0)
total_positive =data.copy()
total_positive = total_positive.loc[total_positive['rate'] == 'positive']
total_positive=total_positive[:3000]
train_pos,test_pos = train_test_split(total_positive,train_size=0.5,random_state=0)
data_copy = pd.concat([total_negative, total_positive], axis=0)
data_train = pd.concat([train_neg, train_pos], axis=0)
data_test= pd.concat([test_neg, test_pos], axis=0)

In [ ]:
print("Size of train dataset: ",data_train.shape)
print("Size of test dataset: ",data_test.shape)

Size of train dataset:  (3000, 2)
Size of test dataset:  (3000, 2)


In [ ]:
train_df = data_train

eval_df =  data_test

In [ ]:
# text.texts_from_df return two tuples
# maxlen means it is considering that much words and rest are getting trucated
# preprocess_mode means tokenizing, embedding and transformation of text corpus(here it is considering BERT model)
(X_train, y_train), (X_test, y_test), preproc = text.texts_from_df(train_df= train_df,
                                                                   text_column = 'Review',
                                                                   label_columns = 'rate',
                                                                   val_df = eval_df,
                                                                   maxlen = 500,
                                                                   preprocess_mode = 'bert')

In [ ]:
# name = "bert" means, here we are using BERT model.

model = text.text_classifier(name = 'bert',
                             train_data = (X_train, y_train),
                             preproc = preproc)

Is Multi-Label? False
maxlen is 500
done.


In [ ]:
learner = ktrain.get_learner(model=model, train_data=(X_train, y_train), batch_size = 6)

In [ ]:
#Essentially fit is a very basic training loop, whereas fit one cycle uses the one cycle policy callback

learner.fit_onecycle(lr = 2e-5, epochs = 4)

predictor = ktrain.get_predictor(learner.model, preproc)
predictor.save('/content/drive/My Drive/bert8')



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/4
500/500 [==============================] - 491s 950ms/step - loss: 0.6158 - accuracy: 0.6376
Epoch 2/4
500/500 [==============================] - 478s 957ms/step - loss: 0.3347 - accuracy: 0.8615
Epoch 3/4
500/500 [==============================] - 478s 956ms/step - loss: 0.1801 - accuracy: 0.9334
Epoch 4/4
500/500 [==============================] - 478s 955ms/step - loss: 0.0458 - accuracy: 0.9863


In [ ]:

data = ['this movie was horrible, the plot was really boring. acting was okay',
        'the fild is really sucked. there is not plot and acting was bad',
        'what a beautiful movie. great plot. acting was good. will see it again']

In [ ]:
predictor.predict(data)

['negative', 'negative', 'positive']

In [ ]:
learner.validate(val_data=(X_test, y_test), class_names=['positive','negative'])

              precision    recall  f1-score   support

    positive       0.87      0.80      0.83       500
    negative       0.82      0.88      0.85       500

    accuracy                           0.84      1000
   macro avg       0.84      0.84      0.84      1000
weighted avg       0.84      0.84      0.84      1000



array([[400, 100],
       [ 59, 441]])